In [1]:
import pandas as pd
pd.options.display.max_columns = None

from scripts import (
    load_uci, load_tidepool_dummy, 
    load_so_pump_raw, load_so_pump_clean,
    load_so_cgm    
)

In [2]:
df = load_uci()
print(df.shape)
df.head()

45 records failed to convert to date
(19490, 5)


,Code,Value,person_id,code_cat,date_time
0,65,0,36,Hypoglycemic symptoms,1991-03-28 15:35:00
1,67,0,36,More-than-usual meal ingestion,1991-03-28 15:35:00
2,56,116,36,NaN,1991-03-28 14:35:00
3,57,39,36,Unspecified blood glucose measurement,1991-04-01 02:18:00
4,65,0,36,Hypoglycemic symptoms,1991-04-01 02:21:00


In [3]:
df = load_tidepool_dummy()
print(df.shape)
df.head()

(2196, 54)


,jsonRowIndex,annotations,clockDriftOffset,conversionOffset,deliveryType,deviceId,deviceTime,duration,expectedDuration,guid,id,payload,percent,rate,scheduleName,suppressed,time,timezoneOffset,type,uploadId,expectedNormal,extended,normal,subType,_deduplicator,units,value,alarmType,reason,status,activeSchedule,basalSchedules,bgTarget,carbRatio,insulinSensitivity,_dataState,_state,byUser,computerTime,dataSetType,deviceManufacturers,deviceModel,deviceSerialNumber,deviceTags,state,timeProcessing,timezone,version,bgInput,bolus,carbInput,insulinCarbRatio,insulinOnBoard,recommended
0,2193,NaN,0.0,0,scheduled,InsOmn-130346997,2017-12-31T18:00:00,14400000.0,NaN,6b6b6e42-abcf-4554-b121-7074a0918484,h16d0och05pod6m1ehr858caibmh0vee,{'logIndices': [1955]},NaN,1.15,basal 3,NaN,2018-01-01T02:00:00.000Z,-480,basal,upid_6badc63b6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2190,NaN,0.0,0,scheduled,InsOmn-130346997,2017-12-31T22:00:00,7200000.0,NaN,cc62b7b6-227c-4bef-9883-b88066063706,u35ijkv5aob5vj8simoefrhic4uv4fie,{'logIndices': [1957]},NaN,1.30,basal 3,NaN,2018-01-01T06:00:00.000Z,-480,basal,upid_6badc63b6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2188,NaN,0.0,0,scheduled,InsOmn-130346997,2018-01-01T00:00:00,12600000.0,NaN,5f37715a-3589-4283-8d88-f9398f14c2d3,2r4utmu6i7il5c8jbhregalm4g45get5,{'logIndices': [1959]},NaN,1.45,basal 3,NaN,2018-01-01T08:00:00.000Z,-480,basal,upid_6badc63b6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2185,NaN,0.0,0,scheduled,InsOmn-130346997,2018-01-01T03:30:00,7200000.0,NaN,f9aeb728-6723-4f3d-8a03-cd8854a459bb,kl7kujnra58rvcad1s7qo98p9410vcg3,{'logIndices': [1962]},NaN,1.30,basal 3,NaN,2018-01-01T11:30:00.000Z,-480,basal,upid_6badc63b6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2184,NaN,0.0,0,scheduled,InsOmn-130346997,2018-01-01T05:30:00,10163000.0,NaN,bab3ca8e-f663-4299-a1c9-60b5b766a868,ia090q89aeu1ap60je5trfksoiel60j9,{'logIndices': [1963]},NaN,1.55,basal 3,NaN,2018-01-01T13:30:00.000Z,-480,basal,upid_6badc63b6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = load_so_pump_raw()
print(df.shape)
df.head()

(4507, 26)


,DATEEVENT,TIMESLOT,EVENTTYPE,VENDOR_EVENT_ID,KEY0,KEY1,KEY2,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,D0,D1,D2,D3,D4,C0,C1,C2,COMMENT
0,2018-08-19 00:00:00,-1,2,NaN,0,12600,2,2,0,0,0,0,0,0,0,0,0,12.6,12.6,0.0,0.0,0.0,NaN,End of Day Basal Total,NaN,NaN
1,2018-08-19 00:00:00,-1,3,NaN,5,16700,0,5,0,0,0,0,0,0,0,0,0,16.7,16.7,0.0,0.0,0.0,Bolus,Bolus,End of Day General Bolus Total,NaN
2,2018-08-19 10:28:58,2,2,0-H1,0,1300,0,0,0,0,0,0,0,0,0,0,0,1.3,0.0,0.0,0.0,0.0,NaN,Pod activated.Basal rate set to 1.30 units/hour.,NaN,Pod activated.Basal rate set to 1.30 units/hour.
3,2018-08-19 10:30:00,2,2,0-H2,0,1000,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,NaN,Basal rate set to 1.00 units/hour.,NaN,Basal rate set to 1.00 units/hour.
4,2018-08-19 10:44:05,2,3,0-H3,2,2000,0,2,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,0.0,0.0,Bolus,Bolus,General Bolus.,Suggested Bolus: ; Programmed Meal: ; Programm...


In [5]:
df = load_so_pump_clean()
print(df.shape)
df.head()

(4507, 7)


,Datetime,Type,Value,Unit,Description,Other Info,Comment
0,2018-08-19 00:00:00,Insulin Summary,12.6,units,End of Day Basal Total,Basal,NaN
1,2018-08-19 00:00:00,Insulin Summary,16.7,units,Bolus-End of Day General Bolus Total,Bolus,NaN
2,2018-08-19 10:28:00,Basal Insulin,1.3,units,Pod activated.Basal rate set to 1.30 units/hour.,Basal,Pod activated.Basal rate set to 1.30 units/hour.
3,2018-08-19 10:30:00,Basal Insulin,1.0,units,Basal rate set to 1.00 units/hour.,Basal,Basal rate set to 1.00 units/hour.
4,2018-08-19 10:44:00,Bolus Insulin,2.0,units,Bolus-General Bolus.,Bolus,Suggested Bolus: ; Programmed Meal: ; Programm...


In [6]:
df = load_so_cgm()
print(df.shape)
df.head()

(25285, 14)


,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
9,10,2019-01-14T00:02:42,EGV,NaN,NaN,NaN,iPhone,128,NaN,NaN,NaN,NaN,1943354.0,41W22H
10,11,2019-01-14T00:07:41,EGV,NaN,NaN,NaN,iPhone,129,NaN,NaN,NaN,NaN,1943654.0,41W22H
11,12,2019-01-14T00:12:41,EGV,NaN,NaN,NaN,iPhone,130,NaN,NaN,NaN,NaN,1943954.0,41W22H
12,13,2019-01-14T00:17:42,EGV,NaN,NaN,NaN,iPhone,131,NaN,NaN,NaN,NaN,1944254.0,41W22H
13,14,2019-01-14T00:22:41,EGV,NaN,NaN,NaN,iPhone,132,NaN,NaN,NaN,NaN,1944554.0,41W22H


In [7]:
# these duplicates are problematic
(df.loc[df['Timestamp (YYYY-MM-DDThh:mm:ss)'].duplicated(keep=False)]
   .sort_values('Timestamp (YYYY-MM-DDThh:mm:ss)')
   .head())

,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
13907,13908,2019-03-03T16:31:28,EGV,NaN,NaN,NaN,iPhone,153,NaN,NaN,NaN,NaN,6146253.0,41W22H
13908,13909,2019-03-03T16:31:28,EGV,NaN,NaN,NaN,iPhone,188,NaN,NaN,NaN,NaN,6149853.0,41W22H
13909,13910,2019-03-03T16:36:28,EGV,NaN,NaN,NaN,iPhone,154,NaN,NaN,NaN,NaN,6146553.0,41W22H
13910,13911,2019-03-03T16:36:28,EGV,NaN,NaN,NaN,iPhone,188,NaN,NaN,NaN,NaN,6150153.0,41W22H
13911,13912,2019-03-03T16:41:27,EGV,NaN,NaN,NaN,iPhone,188,NaN,NaN,NaN,NaN,6150453.0,41W22H
